<a href="https://colab.research.google.com/github/JAEHYEONG8121/hackathon/blob/main/KNN%26Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
!pip install catboost

In [100]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [101]:
train_dir = '/content/drive/MyDrive/CA/train.csv'
test_dir = '/content/drive/MyDrive/CA/test.csv'

In [102]:
import pandas as pd

train_df = pd.read_csv(train_dir)

In [103]:
train_df.shape

(11999, 22)

In [104]:
train_df.describe()

,port_src,port_dst,duration,pkt_count_fwd,pkt_count_bwd,rate_fwd_pkts,rate_bwd_pkts,rate_fwd_bytes,rate_bwd_bytes,payload_fwd_mean,payload_bwd_mean,tcp_win_fwd_init,tcp_win_bwd_init,tcp_syn_count,tcp_psh_count,tcp_rst_count,iat_avg_packets
count,8625.000000,9712.000000,10925.000000,11999.000000,11999.000000,1.082800e+04,1.031200e+04,1.199900e+04,1.199900e+04,10214.000000,10214.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,9.239000e+03
mean,42746.903420,6445.632619,16.712027,16.019168,20.222685,6.645382e+03,6.627536e+03,3.205688e+05,1.887607e+05,183.602231,183.602231,8464.435870,8691.062839,0.713976,2.409451,0.324027,3.409358e+08
std,20898.422967,16426.098766,120.178667,1125.156847,1559.513622,4.940035e+04,5.203941e+04,6.399730e+06,1.183179e+06,312.584458,312.584458,15043.771463,14180.141107,1.010957,13.620653,0.468030,6.284508e+08
min,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,35892.000000,58.000000,0.000037,1.000000,1.000000,0.000000e+00,1.455567e-02,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.738550e-03
50%,51267.000000,83.000000,0.023343,2.000000,2.000000,7.683389e+00,1.477242e+01,2.479247e+01,4.142001e+02,61.500000,61.500000,0.000000,0.000000,0.000000,0.000000,0.000000,4.816914e-02
75%,58154.000000,447.000000,0.363972,6.000000,5.000000,8.485899e+01,5.011536e+02,2.209445e+03,8.446480e+03,141.000000,141.000000,8192.000000,26847.000000,2.000000,2.000000,1.000000,1.954012e+01
max,65535.000000,65384.000000,8567.023224,123229.000000,170796.000000,1.048576e+06,2.097152e+06,5.094846e+08,5.329233e+07,2433.333333,2433.333333,65535.000000,65535.000000,9.000000,1046.000000,1.000000,1.499458e+09


In [105]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11999 entries, 0 to 11998
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                11999 non-null  object 
 1   ip_src            9487 non-null   object 
 2   port_src          8625 non-null   float64
 3   ip_dst            10695 non-null  object 
 4   port_dst          9712 non-null   float64
 5   protocol          11999 non-null  object 
 6   duration          10925 non-null  float64
 7   pkt_count_fwd     11999 non-null  int64  
 8   pkt_count_bwd     11999 non-null  int64  
 9   rate_fwd_pkts     10828 non-null  float64
 10  rate_bwd_pkts     10312 non-null  float64
 11  rate_fwd_bytes    11999 non-null  float64
 12  rate_bwd_bytes    11999 non-null  float64
 13  payload_fwd_mean  10214 non-null  float64
 14  payload_bwd_mean  10214 non-null  float64
 15  tcp_win_fwd_init  11999 non-null  int64  
 16  tcp_win_bwd_init  11999 non-null  int64 

In [106]:
train_df['attack_type'].value_counts()

,count
attack_type,
Benign,8791
Hulk,1719
Port_Scanning,793
DDoS,471
FTP_Brute_Force,47
GoldenEye,41
Slow_HTTP,34
SSH_Brute_Force,30
Botnet,27


In [107]:
train_df.isna().sum()

,0
ID,0
ip_src,2512
port_src,3374
ip_dst,1304
port_dst,2287
protocol,0
duration,1074
pkt_count_fwd,0
pkt_count_bwd,0
rate_fwd_pkts,1171


In [108]:
import ipaddress
# 사설 IP 여부 판단 함수
def is_private_ip(ip):
    try:
        return int(ipaddress.ip_address(ip).is_private)
    except:
        return None  # IP가 잘못됐거나 null일 때

# IP 컬럼 처리 함수
def process_ip_column(df, col):
    # 사설 여부 컬럼 생성
    df[f'{col}_private'] = df[col].apply(is_private_ip)

    # 최빈값 구해서 결측치 대체
    mode = df[f'{col}_private'].mode()
    if not mode.empty:
        df[f'{col}_private'] = df[f'{col}_private'].fillna(mode[0])
    else:
        df[f'{col}_private'] = df[f'{col}_private'].fillna(0)  # fallback

    return df

# 적용
train_df = process_ip_column(train_df, 'ip_src')
train_df = process_ip_column(train_df, 'ip_dst')
train_df = train_df.drop(columns=['ip_src', 'ip_dst'])

In [109]:
print(train_df.isnull().sum())

ID                     0
port_src            3374
port_dst            2287
protocol               0
duration            1074
pkt_count_fwd          0
pkt_count_bwd          0
rate_fwd_pkts       1171
rate_bwd_pkts       1687
rate_fwd_bytes         0
rate_bwd_bytes         0
payload_fwd_mean    1785
payload_bwd_mean    1785
tcp_win_fwd_init       0
tcp_win_bwd_init       0
tcp_syn_count          0
tcp_psh_count          0
tcp_rst_count          0
iat_avg_packets     2760
attack_type            0
ip_src_private         0
ip_dst_private         0
dtype: int64


In [110]:
train_df.head()

,ID,port_src,port_dst,protocol,duration,pkt_count_fwd,pkt_count_bwd,rate_fwd_pkts,rate_bwd_pkts,rate_fwd_bytes,...,payload_bwd_mean,tcp_win_fwd_init,tcp_win_bwd_init,tcp_syn_count,tcp_psh_count,tcp_rst_count,iat_avg_packets,attack_type,ip_src_private,ip_dst_private
0,TRAIN_00000,3721.0,55.0,UDP,0.000231,2,2,8656.974200,8656.974200,259709.226006,...,81.000000,0,0,0,0,0,NaN,Benign,1.0,1.0
1,TRAIN_00001,NaN,91.0,TCP,0.000000,0,1,0.000000,0.000000,0.000000,...,0.000000,0,16392,0,0,0,1.499097e+09,Benign,1.0,1.0
2,TRAIN_00002,NaN,83.0,TCP,0.606002,11,5,18.151760,8.250800,1727.717487,...,790.125000,29200,28960,2,4,1,4.040012e-02,Hulk,1.0,1.0
3,TRAIN_00003,47668.0,NaN,TCP,1.003829,6,6,5.977114,5.977114,324.756507,...,993.416667,29200,28960,3,2,0,9.125718e-02,Hulk,1.0,1.0
4,TRAIN_00004,51753.0,451.0,TCP,181.195271,62,89,NaN,0.491183,6.573019,...,NaN,8192,29200,2,22,0,NaN,Benign,1.0,0.0


In [111]:
from sklearn.impute import KNNImputer

In [112]:
# 숫자형 컬럼만 선택 (KNN은 수치형만 가능)
numeric_cols = train_df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# 라벨(attack_type)은 제외
numeric_cols = [col for col in numeric_cols if col != 'attack_type']

# 해당 부분만 추출
data_for_impute = train_df[numeric_cols]

# KNN 기반 결측치 대체
imputer = KNNImputer(n_neighbors=5)  # 유사한 5개 이웃 기준 평균
imputed_data = imputer.fit_transform(data_for_impute)

# 다시 DataFrame으로
imputed_df = pd.DataFrame(imputed_data, columns=numeric_cols)

# 원본에 덮어쓰기
train_df[numeric_cols] = imputed_df

In [113]:
#print(train_df.isnull().sum())
#print(train_df.columns.tolist())
print(train_df[train_df['attack_type'] == 9])


Empty DataFrame
Columns: [ID, port_src, port_dst, protocol, duration, pkt_count_fwd, pkt_count_bwd, rate_fwd_pkts, rate_bwd_pkts, rate_fwd_bytes, rate_bwd_bytes, payload_fwd_mean, payload_bwd_mean, tcp_win_fwd_init, tcp_win_bwd_init, tcp_syn_count, tcp_psh_count, tcp_rst_count, iat_avg_packets, attack_type, ip_src_private, ip_dst_private]
Index: []

[0 rows x 22 columns]


In [114]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
import pandas as pd
import numpy as np

# 1. 불필요한 컬럼 제거
train_df = train_df.drop(columns=['ID'])

# 2. 라벨 인코딩
attack_label_encoder = LabelEncoder()
protocol_label_encoder = LabelEncoder()
train_df['attack_type'] = attack_label_encoder.fit_transform(train_df['attack_type'])
train_df['protocol'] = protocol_label_encoder.fit_transform(train_df['protocol'])

# 3. 분리
y = train_df['attack_type']
X = train_df.drop(columns=['attack_type'])

# 4. 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 5. 최종 테스트셋 먼저 20% 분리
X_train_val, X_final_test, y_train_val, y_final_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_scaled, y)):
  print(f"\n Fold {fold+1} 시작")

  X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
  y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

  print(f"학습 클래스 분포:\n{y_train.value_counts().sort_index()}")
  print(f"검증 클래스 분포:\n{y_val.value_counts().sort_index()}")


# 5. 첫 번째 분할: 8:2 (학습+검증 : 최종테스트)
#X_train_val, X_final_test, y_train_val, y_final_test = train_test_split(
#    X_scaled, y, test_size=0.2, random_state=42
#)

# 6. 두 번째 분할: 8을 다시 8:2로 (학습 : 검증)
#X_train, X_val, y_train, y_val = train_test_split(
#    X_train_val, y_train_val, test_size=0.2, random_state=42
#)

print("데이터 분할 결과:")
print(f"학습 데이터:     {X_train.shape[0]} samples ({X_train.shape[0]/len(X_scaled)*100:.1f}%)")
print(f"검증 데이터:     {X_val.shape[0]} samples ({X_val.shape[0]/len(X_scaled)*100:.1f}%)")
print(f"최종 테스트:     {X_final_test.shape[0]} samples ({X_final_test.shape[0]/len(X_scaled)*100:.1f}%)")

# 학습 데이터 클래스 분포 확인
train_class_counts = y_train.value_counts().sort_index()
print(f"\n학습 데이터 클래스 분포:")
print(train_class_counts)

# 100개 이하 클래스 확인
minority_classes = train_class_counts[train_class_counts <= 100].index
print(f"\n100개 이하 클래스: {minority_classes.tolist()}")
print(f"해당 클래스 개수: {len(minority_classes)}개")


 Fold 1 시작
학습 클래스 분포:
attack_type
0     7032
1       22
2      376
3       38
4       33
5     1376
6      634
7       24
8       27
9       21
10      11
11       5
Name: count, dtype: int64
검증 클래스 분포:
attack_type
0     1759
1        5
2       95
3        9
4        8
5      343
6      159
7        6
8        7
9        5
10       3
11       1
Name: count, dtype: int64

 Fold 2 시작
학습 클래스 분포:
attack_type
0     7033
1       22
2      377
3       37
4       33
5     1375
6      635
7       24
8       27
9       20
10      12
11       4
Name: count, dtype: int64
검증 클래스 분포:
attack_type
0     1758
1        5
2       94
3       10
4        8
5      344
6      158
7        6
8        7
9        6
10       2
11       2
Name: count, dtype: int64

 Fold 3 시작
학습 클래스 분포:
attack_type
0     7033
1       21
2      377
3       37
4       33
5     1375
6      635
7       24
8       27
9       21
10      11
11       5
Name: count, dtype: int64
검증 클래스 분포:
attack_type
0     1758
1        6
2       94
3  

In [115]:
from imblearn.over_sampling import SMOTE
from collections import Counter

print("="*60)
print("소수 클래스 보수적 증강 (SMOTE)")
print("="*60)

# 현재 클래스 분포
original_distribution = Counter(y_train)
print("원본 분포:")
for class_id, count in sorted(original_distribution.items()):
    print(f"  클래스 {class_id}: {count}개")

# 타겟 분포 계산 (100개 이하 클래스만 3배 증강)
target_distribution = {}
for class_id, count in original_distribution.items():
    if class_id == 11:
        target_distribution[class_id] = min(count * 8, 300)
    elif class_id in minority_classes and count <= 100:
        target_distribution[class_id] = min(count * 5, 300)  # 최대 300개로 제한
    else:
        target_distribution[class_id] = count

print(f"\n타겟 분포 (100개 이하 클래스 3배 증강):")
for class_id, count in sorted(target_distribution.items()):
    original_count = original_distribution[class_id]
    if count > original_count:
        print(f"  클래스 {class_id}: {original_count} → {count}개 (+{count-original_count})")
    else:
        print(f"  클래스 {class_id}: {count}개")

# SMOTE 적용
smote = SMOTE(
    sampling_strategy=target_distribution,
    random_state=42,
    k_neighbors=4  # 보수적 설정
)

X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# 결과 확인
final_distribution = Counter(y_train_balanced)
print(f"\n증강 후 분포:")
total_added = 0
for class_id, count in sorted(final_distribution.items()):
    original_count = original_distribution[class_id]
    added = count - original_count
    total_added += added
    if added > 0:
        print(f"  클래스 {class_id}: {count}개 (+{added})")
    else:
        print(f"  클래스 {class_id}: {count}개")

print(f"\n총 증강된 샘플: {total_added}개")
print(f"최종 학습 데이터: {len(X_train_balanced)}개")

소수 클래스 보수적 증강 (SMOTE)
원본 분포:
  클래스 0: 7033개
  클래스 1: 22개
  클래스 2: 377개
  클래스 3: 38개
  클래스 4: 33개
  클래스 5: 1375개
  클래스 6: 634개
  클래스 7: 24개
  클래스 8: 27개
  클래스 9: 21개
  클래스 10: 11개
  클래스 11: 5개

타겟 분포 (100개 이하 클래스 3배 증강):
  클래스 0: 7033개
  클래스 1: 22 → 110개 (+88)
  클래스 2: 377개
  클래스 3: 38 → 190개 (+152)
  클래스 4: 33 → 165개 (+132)
  클래스 5: 1375개
  클래스 6: 634개
  클래스 7: 24 → 120개 (+96)
  클래스 8: 27 → 135개 (+108)
  클래스 9: 21 → 105개 (+84)
  클래스 10: 11 → 55개 (+44)
  클래스 11: 5 → 40개 (+35)

증강 후 분포:
  클래스 0: 7033개
  클래스 1: 110개 (+88)
  클래스 2: 377개
  클래스 3: 190개 (+152)
  클래스 4: 165개 (+132)
  클래스 5: 1375개
  클래스 6: 634개
  클래스 7: 120개 (+96)
  클래스 8: 135개 (+108)
  클래스 9: 105개 (+84)
  클래스 10: 55개 (+44)
  클래스 11: 40개 (+35)

총 증강된 샘플: 739개
최종 학습 데이터: 10339개


In [116]:
from sklearn.utils.class_weight import compute_class_weight

# 증강된 데이터 기준으로 클래스 가중치 계산
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_balanced),
    y=y_train_balanced
)

class_weight_dict = dict(enumerate(class_weights))
print("클래스 가중치 (증강 후):")
for class_id, weight in sorted(class_weight_dict.items()):
    print(f"  클래스 {class_id}: {weight:.4f}")

# 샘플 가중치 생성
sample_weights = pd.Series(y_train_balanced).map(class_weight_dict)
n_classes = len(np.unique(y_train_balanced))

클래스 가중치 (증강 후):
  클래스 0: 0.1225
  클래스 1: 7.8326
  클래스 2: 2.2854
  클래스 3: 4.5346
  클래스 4: 5.2217
  클래스 5: 0.6266
  클래스 6: 1.3590
  클래스 7: 7.1799
  클래스 8: 6.3821
  클래스 9: 8.2056
  클래스 10: 15.6652
  클래스 11: 21.5396


In [125]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, accuracy_score
from xgboost import callback
from sklearn.utils.class_weight import compute_class_weight

print("="*60)
print("Individual Model Training (증강된 데이터)")
print("="*60)

# XGBoost
print("\nTraining XGBoost...")
xgb_model = XGBClassifier(
    n_estimators=1000,
    max_depth=4,
    learning_rate=0.05,
    objective='multi:softprob',
    num_class=n_classes,
    callbacks=[
        callback.EarlyStopping(rounds=50, min_delta=1e-3),
    ],
    eval_metric='mlogloss',
    use_label_encoder=False,
    random_state=42,
    verbosity=0
)
xgb_model.fit(X_train_balanced, y_train_balanced,
              sample_weight=sample_weights,
              eval_set=[(X_val, y_val)],
              verbose=False,
            )

# LightGBM
print("Training LightGBM...")
lgb_model = LGBMClassifier(
    n_estimators=1000,
    max_depth=4,
    learning_rate=0.03,
    objective='multiclass',
    num_class=n_classes,
    early_stopping_rounds=50,
    metric='multi_logloss',
    random_state=42,
    verbosity=-1,
    force_col_wise=True
)
lgb_model.fit(
    X_train_balanced, y_train_balanced,
    sample_weight=sample_weights,
    eval_set=[(X_val, y_val)],
)

# Class 11 binary classification model

# 이진 타겟 생성
y_train_bin_11 = (y_train_balanced == 11).astype(int)
y_val_bin_11 = (y_val == 11).astype(int)
sample_weights_bin11 = y_train_bin_11.map({0: 1.0, 1: 5.0})

print("Training Binary Classifier for Class 11...")

xgb_bin_11 = XGBClassifier(
    n_estimator=300,
    max_depth=3,
    learning_rate=0.05,
    scale_pos_weight=20,
    callbacks=[
        callback.EarlyStopping(rounds=30)
    ],
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42,
    verbosity=0
)

xgb_bin_11.fit(
    X_train_balanced, y_train_bin_11,
    sample_weight=sample_weights_bin11,
    eval_set=[(X_val, y_val_bin_11)],
    verbose=False
)

print("All models trained successfully!")

Individual Model Training (증강된 데이터)

Training XGBoost...
Training LightGBM...


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Training Binary Classifier for Class 11...
All models trained successfully!


In [126]:
from sklearn.metrics import classification_report, accuracy_score, f1_score
import numpy as np

print("\n" + "="*60)
print("Validation Set Performance")
print("="*60)

# ========================
# STEP 1. 개별 모델 예측
# ========================
xgb_val_preds = xgb_model.predict(X_val)
lgb_val_preds = lgb_model.predict(X_val)

# 1차원 배열로 변환
xgb_val_preds = np.array(xgb_val_preds).flatten()
lgb_val_preds = np.array(lgb_val_preds).flatten()

# ========================
# STEP 2. 이진 분류기 예측 및 덮어쓰기
# ========================
bin_probs = xgb_bin_11.predict_proba(X_val)[:, 1]
threshold = 0.8
bin_11_preds = (bin_probs > threshold).astype(int)

# XGBoost + class 11 보정
xgb_val_preds = np.where(bin_11_preds == 1, 11, xgb_val_preds)

# LightGBM + class 11 보정
lgb_val_preds = np.where(bin_11_preds == 1, 11, lgb_val_preds)

# ========================
# STEP 3. 개별 성능 평가
# ========================
models = ['XGBoost + bin11', 'LightGBM + bin11']
predictions = [xgb_val_preds, lgb_val_preds]

individual_scores = []
print("개별 모델 성능:")
for model_name, preds in zip(models, predictions):
    acc = accuracy_score(y_val, preds)
    f1_macro = f1_score(y_val, preds, average='macro')
    f1_weighted = f1_score(y_val, preds, average='weighted')

    individual_scores.append(f1_macro)
    print(f"{model_name:20} | Accuracy: {acc:.4f} | Macro F1: {f1_macro:.4f} | Weighted F1: {f1_weighted:.4f}")
    print("\n[Classification Report]")
    print(classification_report(y_val, preds, digits=4))
    print("-" * 60)

# ========================
# STEP 4. 앙상블 평가
# ========================
print(f"\n앙상블 성능:")

# 1. Hard Voting
simple_ensemble = []
for i in range(len(xgb_val_preds)):
    votes = [xgb_val_preds[i], lgb_val_preds[i]]
    simple_ensemble.append(max(set(votes), key=votes.count))
simple_ensemble = np.array(simple_ensemble)

# 2. Weighted Voting
total_score = sum(individual_scores)
weights = [score / total_score for score in individual_scores]
print(f"성능 기반 가중치: XGB={weights[0]:.3f}, LGB={weights[1]:.3f}")

weighted_ensemble = []
for i in range(len(xgb_val_preds)):
    weighted_vote = (weights[0] * xgb_val_preds[i] + weights[1] * lgb_val_preds[i])
    weighted_ensemble.append(int(round(weighted_vote)))
weighted_ensemble = np.array(weighted_ensemble)

# 3. Soft Voting (확률 기반) + 이진 분류기 반영
xgb_val_proba = xgb_model.predict_proba(X_val)
lgb_val_proba = lgb_model.predict_proba(X_val)
avg_proba = (xgb_val_proba + lgb_val_proba) / 2
soft_ensemble = np.argmax(avg_proba, axis=1)
soft_ensemble = np.where(bin_11_preds == 1, 11, soft_ensemble)

# ========================
# STEP 5. 앙상블 성능 평가
# ========================
ensemble_methods = ['Hard Voting', 'Weighted (Performance)', 'Soft Voting']
ensemble_preds = [simple_ensemble, weighted_ensemble, soft_ensemble]

val_results = []
for method_name, preds in zip(ensemble_methods, ensemble_preds):
    acc = accuracy_score(y_val, preds)
    f1_macro = f1_score(y_val, preds, average='macro')
    f1_weighted = f1_score(y_val, preds, average='weighted')

    val_results.append({
        'Model': method_name,
        'Accuracy': acc,
        'Macro F1': f1_macro,
        'Weighted F1': f1_weighted
    })

    print(f"{method_name:20} | Accuracy: {acc:.4f} | Macro F1: {f1_macro:.4f} | Weighted F1: {f1_weighted:.4f}")

# ========================
# STEP 6. 최종 선택 및 출력
# ========================
best_ensemble_idx = np.argmax([r['Macro F1'] for r in val_results])
best_ensemble_name = ensemble_methods[best_ensemble_idx]
best_ensemble_preds = ensemble_preds[best_ensemble_idx]

print("\n" + "="*60)
print("📌 최종 선택된 앙상블 평가")
print("="*60)
print(classification_report(y_val, best_ensemble_preds, digits=4))
print(f"\n최고 성능 앙상블: {best_ensemble_name}")
print(f"Best Validation Macro F1: {val_results[best_ensemble_idx]['Macro F1']:.4f}")



Validation Set Performance


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


개별 모델 성능:
XGBoost + bin11      | Accuracy: 0.9875 | Macro F1: 0.8141 | Weighted F1: 0.9882

[Classification Report]
              precision    recall  f1-score   support

           0     0.9983    0.9909    0.9946      1758
           1     0.8333    1.0000    0.9091         5
           2     0.9892    0.9787    0.9840        94
           3     0.8000    0.8889    0.8421         9
           4     0.7778    0.8750    0.8235         8
           5     0.9687    0.9884    0.9784       344
           6     0.9937    0.9874    0.9905       159
           7     0.8571    1.0000    0.9231         6
           8     0.8571    0.8571    0.8571         7
           9     0.8000    0.8000    0.8000         5
          10     0.3333    0.3333    0.3333         3
          11     0.2000    1.0000    0.3333         1

    accuracy                         0.9875      2399
   macro avg     0.7840    0.8916    0.8141      2399
weighted avg     0.9892    0.9875    0.9882      2399

-----------------

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Hard Voting          | Accuracy: 0.9879 | Macro F1: 0.8346 | Weighted F1: 0.9884
Weighted (Performance) | Accuracy: 0.9862 | Macro F1: 0.8058 | Weighted F1: 0.9870
Soft Voting          | Accuracy: 0.9879 | Macro F1: 0.8358 | Weighted F1: 0.9885

📌 최종 선택된 앙상블 평가
              precision    recall  f1-score   support

           0     0.9971    0.9909    0.9940      1758
           1     0.8333    1.0000    0.9091         5
           2     0.9892    0.9787    0.9840        94
           3     0.7273    0.8889    0.8000         9
           4     0.8750    0.8750    0.8750         8
           5     0.9714    0.9884    0.9798       344
           6     0.9937    0.9874    0.9905       159
           7     1.0000    1.0000    1.0000         6
           8     0.7778    1.0000    0.8750         7
           9     1.0000    0.8000    0.8889         5
          10     0.5000    0.3333    0.4000         3
          11     0.2000    1.0000    0.3333         1

    accuracy                      

In [127]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np

print("\n" + "="*60)
print("Final Test Set Performance")
print("="*60)

# 1. 개별 모델 원 예측
xgb_test_preds = xgb_model.predict(X_final_test)
lgb_test_preds = lgb_model.predict(X_final_test)
bin_11_test_preds = xgb_bin_11.predict(X_final_test)

xgb_test_preds = np.array(xgb_test_preds).flatten()
lgb_test_preds = np.array(lgb_test_preds).flatten()

# 2. 이진 분류기 확률 예측
bin_probs_test = xgb_bin_11.predict_proba(X_final_test)[:, 1]
threshold = 0.7

# 3. 모델별로 이진 분류기 결과 적용 (확률 기반으로 class 11 덮어쓰기)
xgb_test_preds = np.where(bin_probs_test > threshold, 11, xgb_test_preds)
lgb_test_preds = np.where(bin_probs_test > threshold, 11, lgb_test_preds)

# 4. 개별 모델 성능 평가
models = ['XGBoost', 'LightGBM']
predictions = [xgb_test_preds, lgb_test_preds]
individual_test_results = []

print("개별 모델 성능:")
for model_name, preds in zip(models, predictions):
    acc = accuracy_score(y_final_test, preds)
    f1_macro = f1_score(y_final_test, preds, average='macro')
    f1_weighted = f1_score(y_final_test, preds, average='weighted')

    individual_test_results.append({
        'Model': model_name,
        'Accuracy': acc,
        'Macro F1': f1_macro,
        'Weighted F1': f1_weighted
    })

    print(f"{model_name:15} | Accuracy: {acc:.4f} | Macro F1: {f1_macro:.4f} | Weighted F1: {f1_weighted:.4f}")
    print(classification_report(y_final_test, preds, digits=4))
    print("-" * 60)

# 5. 앙상블 예측
print(f"\n앙상블 성능:")

# Hard Voting
simple_test_ensemble = []
for i in range(len(xgb_test_preds)):
    votes = [xgb_test_preds[i], lgb_test_preds[i]]
    simple_test_ensemble.append(max(set(votes), key=votes.count))
simple_test_ensemble = np.array(simple_test_ensemble)

# Weighted Voting (가중치는 검증 성능 기준으로 미리 정의되어 있어야 함)
# 예시: weights = [0.33, 0.34, 0.33] 또는 이전 validation 기반 가중치 그대로 사용
weighted_test_ensemble = []
for i in range(len(xgb_test_preds)):
    vote = (weights[0] * xgb_test_preds[i] +
            weights[1] * lgb_test_preds[i])
    weighted_test_ensemble.append(int(round(vote)))
weighted_test_ensemble = np.array(weighted_test_ensemble)

# Soft Voting (확률 예측 후 평균)
xgb_test_proba = xgb_model.predict_proba(X_final_test)
lgb_test_proba = lgb_model.predict_proba(X_final_test)

# 확률 평균
avg_test_proba = (xgb_test_proba + lgb_test_proba) / 2
soft_test_ensemble = np.where(bin_probs_test > threshold, 11, np.argmax(avg_test_proba, axis=1))

# 앙상블 성능 평가
ensemble_methods = ['Hard Voting', 'Weighted Voting', 'Soft Voting']
test_ensemble_preds = [simple_test_ensemble, weighted_test_ensemble, soft_test_ensemble]
test_results = []

for method_name, preds in zip(ensemble_methods, test_ensemble_preds):
    acc = accuracy_score(y_final_test, preds)
    f1_macro = f1_score(y_final_test, preds, average='macro')
    f1_weighted = f1_score(y_final_test, preds, average='weighted')

    test_results.append({
        'Model': method_name,
        'Accuracy': acc,
        'Macro F1': f1_macro,
        'Weighted F1': f1_weighted
    })

    print(f"{method_name:20} | Accuracy: {acc:.4f} | Macro F1: {f1_macro:.4f} | Weighted F1: {f1_weighted:.4f}")

# 6. 최종 성능 비교
best_test_f1 = max([r['Macro F1'] for r in test_results])
best_individual_f1 = max([r['Macro F1'] for r in individual_test_results])
print(f"\nBest Individual Test F1: {best_individual_f1:.4f}")
print(f"Best Ensemble Test F1: {best_test_f1:.4f}")

if best_test_f1 > best_individual_f1:
    print(f"앙상블이 개별 모델보다 {(best_test_f1 - best_individual_f1):.4f} 향상!")
else:
    print("앙상블이 개별 모델보다 성능 향상이 없음")

# 7. 최종 최적 앙상블 결과 Classification Report
best_ensemble_idx = np.argmax([r['Macro F1'] for r in test_results])
final_test_preds = test_ensemble_preds[best_ensemble_idx]

print("\n" + "="*60)
print("📌 최종 선택된 앙상블 결과 Classification Report")
print("="*60)
print(f"Best Ensemble Method: {ensemble_methods[best_ensemble_idx]}")
print(classification_report(y_final_test, final_test_preds, digits=4))



Final Test Set Performance


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


개별 모델 성능:
XGBoost         | Accuracy: 0.9933 | Macro F1: 0.8908 | Weighted F1: 0.9939
              precision    recall  f1-score   support

           0     1.0000    0.9926    0.9963      1758
           1     1.0000    1.0000    1.0000         5
           2     1.0000    1.0000    1.0000        94
           3     1.0000    1.0000    1.0000        10
           4     1.0000    1.0000    1.0000         8
           5     0.9690    1.0000    0.9843       344
           6     1.0000    1.0000    1.0000       159
           7     1.0000    1.0000    1.0000         6
           8     1.0000    0.8571    0.9231         7
           9     1.0000    1.0000    1.0000         5
          10     1.0000    0.3333    0.5000         3
          11     0.1667    1.0000    0.2857         1

    accuracy                         0.9933      2400
   macro avg     0.9280    0.9319    0.8908      2400
weighted avg     0.9952    0.9933    0.9939      2400

-----------------------------------------------

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Hard Voting          | Accuracy: 0.9958 | Macro F1: 0.9019 | Weighted F1: 0.9962
Weighted Voting      | Accuracy: 0.9929 | Macro F1: 0.8821 | Weighted F1: 0.9933
Soft Voting          | Accuracy: 0.9950 | Macro F1: 0.8961 | Weighted F1: 0.9954

Best Individual Test F1: 0.8963
Best Ensemble Test F1: 0.9019
앙상블이 개별 모델보다 0.0056 향상!

📌 최종 선택된 앙상블 결과 Classification Report
Best Ensemble Method: Hard Voting
              precision    recall  f1-score   support

           0     1.0000    0.9954    0.9977      1758
           1     1.0000    1.0000    1.0000         5
           2     1.0000    1.0000    1.0000        94
           3     1.0000    1.0000    1.0000        10
           4     1.0000    1.0000    1.0000         8
           5     0.9829    1.0000    0.9914       344
           6     1.0000    1.0000    1.0000       159
           7     1.0000    1.0000    1.0000         6
           8     1.0000    1.0000    1.0000         7
           9     1.0000    1.0000    1.0000         5
  

In [120]:
test_df = pd.read_csv(test_dir)

In [121]:
# 2. IP 처리
test_df = process_ip_column(test_df, 'ip_src')
test_df = process_ip_column(test_df, 'ip_dst')
test_df = test_df.drop(columns=['ip_src', 'ip_dst'])

# 3. KNN 결측치 대체
test_numeric_cols = test_df.select_dtypes(include=['float64', 'int64']).columns.tolist()
test_numeric_cols = [col for col in test_numeric_cols if col != 'attack_type'] if 'attack_type' in test_df.columns else test_numeric_cols

test_data_for_impute = test_df[test_numeric_cols]
test_imputed_data = imputer.transform(test_data_for_impute)
test_imputed_df = pd.DataFrame(test_imputed_data, columns=test_numeric_cols)
test_df[test_numeric_cols] = test_imputed_df

# 4. 'ID' 제거 (있다면)
if 'ID' in test_df.columns:
    test_df = test_df.drop(columns=['ID'])

# 5. protocol 라벨 인코딩
test_df['protocol'] = protocol_label_encoder.transform(test_df['protocol'])

# 6. attack_type 제거 (있다면)
if 'attack_type' in test_df.columns:
    test_df = test_df.drop(columns=['attack_type'])

# 7. 스케일링
X_test_scaled = scaler.transform(test_df)

In [122]:
from scipy.special import softmax
from scipy.stats import mode
from sklearn.preprocessing import OneHotEncoder

# soft voting을 위한 확률 예측
#test_preds_proba = np.zeros((X_test_scaled.shape[0], len(np.unique(y))))

#proba = lgb_model.predict_proba(X_test_scaled)
#test_preds_proba += proba


# 1. 모델별 확률 예측 (Soft Voting)
xgb_test_proba = xgb_model.predict_proba(X_test_scaled)
lgb_test_proba = lgb_model.predict_proba(X_test_scaled)

# 2. 확률 평균
avg_test_proba = (xgb_test_proba + lgb_test_proba) / 2

# 평균 확률로 클래스 선택
test_preds_final = np.argmax(avg_test_proba, axis=1)

# 라벨 디코딩
test_pred_labels = attack_label_encoder.inverse_transform(test_preds_final)
print(test_pred_labels)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


['Benign' 'Port_Scanning' 'Benign' ... 'Benign' 'Slowloris' 'Benign']


In [123]:
# sample_submission 불러오기
sample_submission = pd.read_csv('/content/drive/MyDrive/CA/sample_submission.csv')

# 예측 결과 삽입
sample_submission['attack_type'] = test_pred_labels.astype(str)

# 저장
sample_submission.to_csv('/content/drive/MyDrive/CA/baseline_submission.csv', index=False)

print("✅ submission 파일 저장 완료!")


✅ submission 파일 저장 완료!
